In [ ]:
# 지금까지 배운 신경망 학습의 절차는 다음과 같다.
'''
- 전제
신경망에는 적응 가능한 가중치와 편향이 있고, 이 가중치와 편향을 훈련 데이터에 적응하도록 조정하는 과정을 '학습'이라고 한다. 신경망 학습은 다음의 4단계로 수행한다.
- 1단계 : 미니배치
훈련 데이터 중 일부를 무작위로 가져온다. 이렇게 선별한 데이터를 미니배치라 하고, 그 미니배치의 손실함수 값을 줄이는 것이 목표이다.
- 2단계 : 기울기 산출
미니배치의 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구한다. 기울기는 손실 함수의 값을 가장 작게하는 방향을 제시한다.
가중치 매개변수의 기울기를 구하면, 매개변수를 조정하였을 때 손실 함수가 어떤 방향으로 바뀔 것인지 알 수 있기 때문이다.
- 3단계 : 매개변수 갱신
가중치 매개변수를 기울기 방향으로 아주 조금 갱신한다.
- 4단계 : 반복
1~3단계를 반복한다.
'''

# 이는 경사 하강법으로 매개변수를 갱신하는 방법이고, 이때 데이터를 미니배치로 무작위로 선정하므로 확률적 경사 하강법stochastic gradient descent,SGD 라고 부른다.

In [15]:
# 손글씨 숫자를 학습하는 신경망을 구현해보자. 2층 신경망(은닉층 1개)을 대상으로 MNIST 데이터셋을 사용하여 학습한다.

import sys, os
sys.path.append(os.pardir)
from common.functions import *
from common.gradient import numerical_gradient

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01): # input_size: 입력층 뉴런수, hidden_size: 은닉층 뉴런수, output_size: 출력층 뉴런수
        # 가중치 초기화
        self.params = {} # 신경망의 매개변수(가중치, 편향)을 보관하는 딕셔너리 변수
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
    def predict(self, x): # 인수 x는 이미지 데이터
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
    
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grad = {} # 기울기를 보관하는 딕셔너리 변수
        grad['W1'] = numerical_gradient(loss_W, self.params['W1']) # 1번째 층의 가중치의 기울기
        grad['b1'] = numerical_gradient(loss_W, self.params['b1']) # 1번째 층의 편향의 기울기
        grad['W2'] = numerical_gradient(loss_W, self.params['W2']) # 2번쨰 층의 가중치의 기울기
        grad['b2'] = numerical_gradient(loss_W, self.params['b2']) # 2번째 층의 편향의 기울기
        
        return grad

In [4]:
# TwoLayerNet 클래스의 params 변수
net = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)
print(net.params['W1'].shape) # (784, 100)
print(net.params['b1'].shape) # (100,)
print(net.params['W2'].shape) # (100, 10)
print(net.params['b2'].shape) # (10, )

(784, 100)
(100,)
(100, 10)
(10,)


In [6]:
# 예측 처리
x = np.random.rand(100, 784) # 더미 입력 데이터(100장 분량)
y = net.predict(x)

In [ ]:
x = np.random.rand(100, 784) # 더미 입력 데이터(100장 분량)
t = np.random.rand(100, 10) # 더미 입력 데이터(100장 분량)

grads = net.numerical_gradient(x, t) # 기울기 계산 결과가 grads에 저장된다.

print(grads['W1'].shape) # (784, 100)
print(grads['b1'].shape) # (100, )
print(grads['W2'].shape) # (100, 10)
print(grads['b2'].shape) # (10, )

In [ ]:
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

train_loss_list = []

# 하이퍼파라미터
iters_num = 10000 # 반복횟수
train_size = x_train.shape[0]
batch_size = 100 # 미니배치 크기
learning_rate = 0.1

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for _ in range(iters_num):
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size) # train 데이터셋에서 batch_size개를 무작위 추출
    x_batch = x_train[batch_mask] # 무작위 추출한 미니배치에 해당하는 입력값을 x_batch에 할당
    t_batch = t_train[batch_mask] # 무작위 추출한 미니배치에 해당하는 정답값을 t_batch에 할당
    
    # 기울기 계산
    grad = network.numerical_gradient(x_batch, t_batch)
    # grad = network.gradient(x_batch, t_batch) #성능 개선판!
    
    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key] # 학습률 * 경사값으로 매개변수를 iters_num번 갱신
    
    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

In [17]:
# 에폭epoch은 하나의 단위로, 1에폭은 학습에서 훈련 데이터를 모두 소진했을 떄의 횟수에 해당한다.
# 예컨대 훈련 데이터 10,000개를 100개의 미니배치로 학습할 경우, 확률적 경사 하강법을 100회 반복하면 모든 훈련 데이터를 '소진'한 게 된다. 이 경우 100회가 1에폭이 된다.

In [ ]:
# 시험데이터로 평가하기

# 1에폭마다 모든 훈련 데이터와 시험 데이터에 대한 정확도를 계산하고, 그 결과를 기록한다.
# 훈련 데이터와 시험 데이터의 정확도가 차이가 나지 않았다면, 오버피팅이 일어나지 않았음을 의미한다(딥러닝이 잘 된 것이다)
# 만약 오버피팅이 일어난다면, 시험 데이터에 대한 정확도가 떨어진다.
# 이 순간을 포착해 학습을 중단하여 오버피팅을 예방하는 것을 '조기 종료early stopping'이라고 하며, '가중치 감소', '드롭아웃'과 함께 대표적인 오버피팅 예방법으로 꼽힌다.

import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

train_loss_list = []
train_acc_list = []
test_acc_lsit = []

# 1에폭당 반복 수
iter_per_epoch = max(train_size / batch_size, 1)

# 하이퍼파라미터
iters_num = 10000 
train_size = x_train.shape[0]
batch_size = 100 
learning_rate = 0.1

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for _ in range(iters_num):
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size) 
    x_batch = x_train[batch_mask] 
    t_batch = t_train[batch_mask] 
    
    # 기울기 계산
    grad = network.numerical_gradient(x_batch, t_batch)
    # grad = network.gradient(x_batch, t_batch) #성능 개선판!
    
    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key] 
    
    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    # 1에폭당 정확도 계산
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(f"train acc, test acc | {train_acc}, {test_acc}")

In [ ]:
# 이번 장에서 배운 내용

# 기계학습에서 사용하는 데이터셋은 훈련 데이터와 시험 데이터로 나뉜다
# 훈련 데이터로 학습한 모델의 범용 능력을 시험 데이터로 평가한다.
# 신경망 학습은 손실 함수를 지표로, 손실 함수의 값이 작아지는 방향으로 가중치 매개변수를 갱신한다.
# 가중치 매개변수를 갱신할 떄는 가중치 매개변수의 기울기를 이용하고, 기울어진 방향으로 가중치의 값을 갱신하는 작업을 반복한다.
# 아주 작은 값을 주었을 때의 차분으로 미분하는 것을 수치 미분이라고 한다.
# 수치 미분을 이용해 가중치 매개변수의 기울기를 구할 수 있다.
# 수치 미분을 이용한 계산에서는 시간이 걸리지만, 그 구현은 간단하다. 한편, 다음 장에서 구현하는 (다소 복잡한) 오차역전파법은 기울기를 고속으로 구할 수 있다.